In [ ]:
# Import modules 
from naive_methods.naive_methods import predict_last_value, predict_mean, predict_random_walk
from ml_models.ml_models import predict_prophet_model
from traditional_models.traditional_models import predict_arima, predict_ets, predict_theta

# Pre-processing 
from data_processing.pre_processing import pre_processing
from data_processing.transform import create_df_with_datetimes

# Scoring
from evaluation_protocol.performance_metrics import mape, smape

# Import libraries
import pandas as pd
import numpy as np 
from configparser import ConfigParser
import os

In [ ]:
m4_train_dataset_df = pd.read_csv('Dataset/Yearly-train.csv')
m4_test_dataset_df = pd.read_csv('Dataset/Yearly-test.csv')
info_df = pd.read_csv('Dataset/M4-info.csv')

In [ ]:
print(len(info_df[info_df['category']=='Macro']))

In [ ]:
mape_list_macro = []
for i in range(80):
    timeseries_train_df = create_df_with_datetimes(m4_train_dataset_df, i)
    timeseries_test_df = create_df_with_datetimes(m4_test_dataset_df, i)
    timeseries_index = m4_test_dataset_df['V1'][i]
    target_column_name = timeseries_test_df.columns[1]
    real = timeseries_test_df[target_column_name]

    predictions, complexity = predict_prophet_model(train=timeseries_train_df, test=timeseries_test_df)
    print(predictions)
    mape_score = mape(y_true=real,y_pred=predictions)

    mape_list_macro.append(mape_score);

In [7]:
def avg(lst):
    return sum(lst) / len(lst)
    
print(avg(mape_list_macro))

0.3936960664874438


In [ ]:
sample_df = pd.read_csv("sample_dataset.csv")

In [ ]:
def allowed_file(filename):
     # Add the allowed file extensions here
    allowed_extensions = set(['csv'])
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in allowed_extensions

In [ ]:
def update_upload_path(filename):
    check_config_object = ConfigParser()
    check_config_object.read("test_config.ini")

    #Get the SINGLESCOREINFO section
    path_config = check_config_object["USERFILE"]
    existing_path = path_config['file_path']

    if filename not in existing_path:
        path_config['file_path'] = os.path.join('uploads', filename)

        #Write changes back to file
        with open('test_config.ini', 'w') as conf:
            check_config_object.write(conf)

In [ ]:
# Upload
def upload_file(uploaded_file):

    # Check if the file is empty
    if uploaded_file == '':
        return 'error: No selected file'

    # Check if the file has an allowed extension
    elif not allowed_file(uploaded_file):
        return f'error Invalid file extension, file extenstion: {uploaded_file.rsplit(".",1)[1].lower()}'
    
    else: # Save the uploaded file to a specific location
        pre_processing_result = pre_processing(uploaded_file)

        if pre_processing_result == True:
            update_upload_path(uploaded_file)
            return 'message File uploaded successfully'
        else: 
            return f'Pre-processing result: {pre_processing_result}'

# UPLOAD

In [ ]:
upload_file('')

In [ ]:
upload_file('test.ini')

In [ ]:
upload_file('sample_dataset.csv')

In [ ]:
upload_file('wrong_column_dataset.csv')

In [ ]:
upload_file('future_values.csv')

In [ ]:
upload_file('invalid_format.csv')

# MODELS

In [ ]:
def split_user_data(filename):

    user_data_df = pd.read_csv(filename)

    # 70-30 split with no fancy means
    split_index = int(0.7 * len(user_data_df))

    train = user_data_df[:split_index]
    test = user_data_df[split_index:]

    return train, test

In [ ]:
train, test = split_user_data('sample_dataset.csv')

In [ ]:
train_last_value = train['target'].tolist()[-1]
train_mean = train['target'].mean()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

def my_arima(series,forecast_periods):
    series = series['target'].astype(float)
    forecast_periods = len(forecast_periods)
    p = 1
    d = 1
    q = 2
    # train_list = train_df.loc[0][1:].dropna().tolist()
    arima_model = ARIMA(series, order=(p,d,q))
    arima_results = arima_model.fit()

    future_predictions = pd.Series(arima_results.forecast(forecast_periods))

    return future_predictions

In [ ]:
from prophet import Prophet
def my_prophet(train, test):
    train.columns = ['ds', 'y']
    test.columns = ['ds', 'y']

    model = Prophet(interval_width=0.95)
    model.fit(train)
    
    future_preds = len(test['ds'])

    # Make forecasts for as many years as the ones included in the test dataset
    future_dates = model.make_future_dataframe(periods=future_preds, freq='YS')

    forecast = model.predict(future_dates)

    forecast_organized = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

    predicted = forecast_organized['yhat'][-future_preds:]

    return predicted

In [ ]:
preds = predict_last_value(train, test)
print(f"Last value: {train_last_value}")
print(f"Last value predictions: {preds}")

print('----------------------')

preds = predict_mean(train, test)
print(f"Mean: {train_mean}")
print(f"Mean predictions: {preds}")

print('----------------------')

preds = predict_random_walk(train, test)
print(f"Last value: {train_last_value}")
print(f"Random walk predictions: {preds}")

print('----------------------')

preds = predict_arima(train, test)
print(f"Arima predictions: {preds}")

print('----------------------')

preds = my_arima(train, test)
print(f"My Arima predictions: {preds}")

print('----------------------')

preds = predict_ets(train, test)
print(f"ETS predictions: {preds}")

print('----------------------')

preds = predict_theta(train, test)
print(f"Theta predictions: {preds}")

print('----------------------')

preds = predict_prophet_model(train, test)
print(f"Prophet predictions: {preds}")

print('----------------------')

preds = my_prophet(train, test)
print(f"My Prophet predictions: {preds}")